In [1]:
# Reference: https://github.com/vlraik/word-level-rnn-keras/blob/master/lstm_text_generation.py

import random
import sys
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

/home/lling086/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lling086/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lling086/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lling086/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame

In [2]:
dir_split = f"../1.DataPreparationResults/obama"

file_train = open(f"{dir_split}/train.txt", 'r').read()
file_val = open(f"{dir_split}/val.txt", 'r').read()
file_test = open(f"{dir_split}/test.txt", 'r').read()

# Add spaces around <speech_sep>
# Create a set of all words in train.txt but remove <speech_sep>
word_train = set(file_train.replace("<speech_sep>", " <speech_sep> ").split())
word_train.remove("<speech_sep>")

print("total number of unique words: ",len(word_train))

word_indices = dict((c, i) for i, c in enumerate(word_train))
indices_word = dict((i, c) for i, c in enumerate(word_train))

total number of unique words:  6508


In [3]:
x_len = 30
x_step = 1

In [4]:
def vectorization(file):
    sentences = []
    sentences2 = []
    next_words = []
    list_words = []

    for speech in file.split("<speech_sep>"):
        list_words = speech.split()
        # I noticed the last speech has zero word 
        # because <speech_sep> is the last character
        if len(list_words) == 0:
            break

        for i in range(0,len(list_words)-x_len, x_step):
            sentences2 = ' '.join(list_words[i: i + x_len])
            sentences.append(sentences2)
            next_words.append(list_words[i + x_len])

    x = np.zeros((len(sentences), x_len, len(word_train)), dtype=np.bool)
    y = np.zeros((len(sentences), len(word_train)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, word in enumerate(sentence.split()):
            # For words that don't exist in train.txt but exist in val.txt or test.txt,
            #     X[i, t] would be all zeros
            if word in word_train:
                x[i, t, word_indices[word]] = 1
        if next_words[i] in word_train:
            y[i, word_indices[next_words[i]]] = 1
            
    return x, y

In [5]:
# Run into memory issue with huge arrays
# Reference: https://stackoverflow.com/questions/57507832/unable-to-allocate-array-with-shape-and-data-type

train_X, train_Y = vectorization(file_train)
print(train_X.shape)
print(train_Y.shape)

val_X, val_Y = vectorization(file_val)
print(val_X.shape)
print(val_Y.shape)

(80001, 30, 6508)
(80001, 6508)
(83061, 30, 6508)
(83061, 6508)


In [6]:
model = keras.Sequential()
model.add(LSTM(512, input_shape=(x_len, len(word_train)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(word_train), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
